In [1]:
import os
from six import moves
import ssl

import tflearn
from tflearn.data_utils import *

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
df = pd.read_csv('villes_france.csv',low_memory=False)
df.sample(5)

,Département,Slug,Nom,Nom simple,Nom reel,Nom soundex,Nom metaphone,Code Postal,Numéro de commune,Code INSEE,...,Densité en 2010,Superficie,Longitude (degré),Latitude (degré),Longitude (GRD),Latitude (GRD),Longitude (DMS),Latitude (DMS),Altitude minimale,Altitude maximale
3586,10,traînel,TRAINEL,trainel,Traînel,T654,TRNL,10400,382,10382,...,52,19.99,3.450000,48.4167,1227.0,53794.0,32627.0,482453.0,66.0,154.0
33773,84,puget,PUGET,puget,Puget,P230,PJT,84360,93,84093,...,37,17.90,5.300000,43.7500,3258.0,48617.0,51609.0,434518.0,123.0,703.0
27806,68,illzach,ILLZACH,illzach,Illzach,I420,ILSX,68110,154,68154,...,1946,7.50,7.333330,47.7833,5568.0,53091.0,72053.0,474656.0,228.0,237.0
19543,51,romery-51,ROMERY,romery,Romery,R560,RMR,51480,465,51465,...,86,2.07,3.916670,49.1000,1754.0,54552.0,35455.0,490550.0,120.0,240.0
34428,86,monthoiron,MONTHOIRON,monthoiron,Monthoiron,M365,MN0RN,86210,164,86164,...,40,16.66,0.616667,46.7333,-1907.0,51926.0,3712.0,464401.0,63.0,141.0


In [3]:
df = df.rename(lambda c : c.rstrip().title().replace(' ','_'),axis=1)
print(*df.columns)

Département Slug Nom Nom_Simple Nom_Reel Nom_Soundex Nom_Metaphone Code_Postal Numéro_De_Commune Code_Insee Arrondissement Canton Unnamed:_12 Population_En_2010 Population_En_1999 Population_En_2012_(Estimée) Densité_En_2010 Superficie Longitude_(Degré) Latitude_(Degré) Longitude_(Grd) Latitude_(Grd) Longitude_(Dms) Latitude_(Dms) Altitude_Minimale Altitude_Maximale


In [4]:
#import sqlite3
#conn = sqlite3.connect('villes_france.sql')
#pd.read_sql('villes_france',conn)

In [5]:
villes = list(df['Nom_Simple'])
all_villes = '\n'.join(sorted(villes))

In [6]:
len(villes)

36700

In [7]:
print(all_villes)

aast
abainville
abancourt
abancourt
abaucourt
abaucourt hautecourt
abbans dessous
abbans dessus
abbaretz
abbecourt
abbecourt
abbenans
abbeville
abbeville la riviere
abbeville les conflans
abbeville saint lucien
abbevillers
abeilhan
abelcourt
abere
abergement la ronce
abergement le grand
abergement le petit
abergement les thesy
abidos
abilly
abitain
abjat sur bandiat
ablain saint nazaire
ablaincourt pressoir
ablainzevelle
ablancourt
ableiges
ablis
ablon
ablon sur seine
aboen
aboncourt
aboncourt
aboncourt gesincourt
aboncourt sur seille
abondance
abondant
abos
abreschviller
abrest
abries
abscon
abzac
abzac
accolans
accolay
accons
accous
achain
achen
achenheim
acheres
acheres
acheres la foret
achery
acheux en amienois
acheux en vimeu
acheville
achey
achicourt
achiet le grand
achiet le petit
achun
achy
acigne
aclou
acon
acoua
acq
acqueville
acqueville
acquigny
acquin westbecourt
acy
acy en multien
acy romance
adaincourt
adainville
adam les passavant
adam les vercel
adamswiller
adast
ade
ad

In [8]:
#context = ssl._create_unverified_context()
#moves.urllib.request.urlretrieve( "https://raw.githubusercontent.com/tflearn/tflearn.github.io/master/resources/US_Cities.txt", path)

In [9]:
maxlen = 30

#cities = open(path, "r").read()#.decode('utf-8')
cities = all_villes
X, Y, char_idx = string_to_semi_redundant_sequences(cities, seq_maxlen=maxlen, redun_step=3)

Vectorizing text...
Text total length: 467,172
Distinct chars   : 30
Total sequences  : 155,714


## Define network

In [10]:
g = tflearn.input_data(shape=[None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 256, return_seq=True)   # Was 512
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 256)    # Was 512
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Training step

In [11]:
#for i in range(1):
seed = random_sequence_from_string(cities, maxlen)
m.fit(X, Y, validation_set=0.1, batch_size=256,
      n_epoch=2, run_id='name_generation')
print("-- TESTING...")
print("-- Test with temperature of 1.2 --")
print(m.generate(30, temperature=1.2, seq_seed=seed).encode('utf-8'))
print("-- Test with temperature of 1.0 --")
print(m.generate(30, temperature=1.0, seq_seed=seed).encode('utf-8'))
print("-- Test with temperature of 0.5 --")
print(m.generate(30, temperature=0.5, seq_seed=seed).encode('utf-8'))

Training Step: 1095  | total loss: 2.19184 | time: 397.666s
| Adam | epoch: 002 | loss: 2.19184 -- iter: 140032/140142
Training Step: 1096  | total loss: 2.18351 | time: 418.871s
| Adam | epoch: 002 | loss: 2.18351 | val_loss: 2.11798 -- iter: 140142/140142
--
INFO:tensorflow:/Users/Alex/Google Drive/GitHub/french-name-generator/model-1096 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'list' object has no attribute 'name'
-- TESTING...
-- Test with temperature of 1.2 --
b'saint germain sur ecole\nsaint tordaud\nwauroudca veuimy molt '
-- Test with temperature of 1.0 --
b'saint germain sur ecole\nsaint noisle res\ndatepnee\nsaitte sai'
-- Test with temperature of

In [12]:
random_sequence_from_string(cities, maxlen).split()

['ge', 'le', 'theil', 'de', 'bretagne', 'le', 'the']

In [13]:
%%bash
tensorboard --logdir=/tmp/tflearn_logs

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W1007 22:47:38.054851 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1007 22:47:38.054851 123145435295744 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1007 22:47:38.060815 Reloader tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1007 22:47:38.060815 123145435295744 tf_logging.py:120] Found more than one metagraph event per run